## Imports

In [19]:
from bs4 import BeautifulSoup
import urllib
import os

In [20]:
import spotipy
import spotipy.util as util

## Parse HTML

In [21]:
def get_beaut_soup(url):
    
    with urllib.request.urlopen(url) as response:
        response = response.read()
    page = BeautifulSoup(response, 'html.parser')
    
    return page

In [22]:
def clean_headliner(headliner):
    if "(" in headliner:
        return headliner.split(" (")[0]
    else:
        return headliner

In [23]:
def parse_r5(url="https://www.r5productions.com/"):
    
    headliners = []
    page = get_beaut_soup(url)
    headliners_html = page.find_all("h1")
    
    headliners = [clean_headliner(headliner.text) for headliner in headliners_html if '\n' not in headliner.text]
    headliners = list(set(headliners))
    return headliners

In [24]:
artists = parse_r5()

## Spotify Authentication

In [25]:
client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']

In [ ]:
username="dan1618"
scope = "playlist-modify-private"

In [36]:
token = util.prompt_for_user_token(username,
                                       scope,
                                       client_id=client_id,
                                       client_secret=client_secret,
                                       redirect_uri="http://localhost:8888/notebooks/spotify_project.ipynb")



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=d6c12a5956614a2b9e06b8d5bffa5722&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fnotebooks%2Fspotify_project.ipynb&scope=playlist-modify-private in your browser




KeyboardInterrupt: 

In [28]:
service = spotipy.Spotify(auth=token)

## Get Tracks

In [29]:
def get_spotify_uri(name, service):
    results = service.search(q="artist:" + name, type="artist")
    uri = ""
    try:
        uri = results["artists"]["items"][0]["uri"]
    except:
        pass
    return uri

In [30]:
def get_top_tracks(uri, service, number=3):
    results = service.artist_top_tracks(uri)
    tracks = [track['id'] for track in results["tracks"][:number]]
    return tracks

## Create Playlist

In [31]:
def make_playlist_from_tracks(username, playlist_name, tracks, service, playlist_description=""):
    playlist = service.user_playlist_create(username, playlist_name, False)
    
    playlist_id = playlist["id"]
    
    service.user_playlist_add_tracks(username, playlist_id, tracks)
    
    return

In [32]:
def make_playlist(service, username, artists, date):
    uris = [get_spotify_uri(artist, service) for artist in artists]
    tracks = [track for uri in uris for track in get_top_tracks(uri, service)]
    
    playlist_name = "R5 Playlist " + date
    make_playlist_from_tracks(username, playlist_name, tracks, service)
    
    return

In [34]:
username

'liis23'

In [33]:
make_playlist(service, username, artists, "Feb 2018")

SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/users/liis23/playlists:
 You cannot create a playlist for another user.